In [27]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, date, time

my_free_api_key = '9a2c24d00b614ff1d74a08f03054ef53'

In [24]:
# get timestamp in correct format for tomorrow 8 AM until 8 PM
def get_tomorrow():
    date_tomorrow = date.today() + timedelta(days=1)
    time_start = time(hour=0, minute=0, second=0)
    time_middle = time(hour=12, minute=0, second=0)
    time_end = time(hour=23, minute=59, second=59)

    datetime_start = datetime.combine(date_tomorrow, time_start).strftime('%Y-%m-%dT%H:%M')
    datetime_middle = datetime.combine(date_tomorrow, time_middle).strftime('%Y-%m-%dT%H:%M')
    datetime_end = datetime.combine(date_tomorrow, time_end).strftime('%Y-%m-%dT%H:%M')
    return datetime_start,datetime_middle,datetime_end

In [34]:
def weather_forcast(list_of_cities):
   
    cities_list  = []
    weather_list = []
    
    for city in list_of_cities :
        city_r = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={my_free_api_key}&units=metric")
        city_j = city_r.json()
        
        cities_list.append(city_j['city'])
        

        for detail in city_j['list']:
            weather_details_dic = {}
            weather_details_dic["city_id"] = city_j['city']['id']
            weather_details_dic["city_name"] = city_j['city']['name']
            weather_details_dic["temp"] = detail['main']['temp']
            weather_details_dic["temp_min"] = detail['main']['temp_min']
            weather_details_dic["temp_max"] = detail['main']['temp_max']
            weather_details_dic["feels_like"] = detail['main']['feels_like']
            weather_details_dic["pressure"] = detail['main']['pressure']
            weather_details_dic["humidity"] = detail['main']['humidity']
            weather_details_dic["weather"] = detail['weather'][0]['main']
            weather_details_dic["weather_description"] = detail['weather'][0]['description']
            weather_details_dic["clouds"] = detail['clouds']['all']
            weather_details_dic["wind_speed"] = detail['wind']['speed']
            weather_details_dic["date_time"] = detail['dt_txt']
            weather_list.append(weather_details_dic)
    
    
    df_weather = pd.DataFrame(weather_list)
    dt_start,dt_middle,dt_end = get_tomorrow()
    df_weather['date_time'] = pd.to_datetime(df_weather['date_time'])
    df_weather = df_weather.loc[lambda df_ : (df_['date_time'] >= dt_start) &  (df_['date_time'] <= dt_end)]
    
    df_cities = pd.json_normalize(cities_list)
    
    
    return df_weather,df_cities

In [35]:
df1, df2 = weather_forcast(['LONDON'])

In [37]:
df2

,id,name,country,population,timezone,sunrise,sunset,coord.lat,coord.lon
0,2643743,London,GB,1000000,3600,1659414327,1659469679,51.5085,-0.1257


In [46]:
def get_citys_airports(city_name, country_iso2):
    
	url = "https://aerodatabox.p.rapidapi.com/airports/search/term"

	querystring = {"q":city_name,"limit":"10"}

	headers = {
		"X-RapidAPI-Key": "5271551a49mshc194f542b6dc600p113722jsn431b32cb69ab",
		"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
	}

	airports_r = requests.request("GET", url, headers=headers, params=querystring)
	airports_j = airports_r.json()

	df_airports =pd.json_normalize(airports_j['items'])
	df_airports = df_airports.loc[lambda df_ : df_['countryCode'] == country_iso2]
	
	return df_airports

In [47]:
temp = get_citys_airports('LONDON','GB')

In [48]:
# right_airports
temp

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EGSS,STN,"London, London Stansted",Stansted,London,GB,51.8850,0.234999
1,EGLL,LHR,"London, London Heathrow",Heathrow,London,GB,51.4706,-0.461941
2,EGLC,LCY,"London, London City",City,London,GB,51.5053,0.055277
3,EGKK,LGW,"London, London Gatwick",Gatwick,London,GB,51.1481,-0.190277
4,EGGW,LTN,"London, London Luton",Luton,London,GB,51.8747,-0.368333
9,EGKB,BQH,"London, London Biggin Hill",Biggin Hill,London,GB,51.3308,0.032499
